# Spatial Data for Reporting

In [163]:
import requests
import pandas as pd
import geopandas as gp
import ee
import json

Helper functions for querying arcgis REST api

In [156]:
def get_geojson_as_json(base_url, path, outFields=None):
    if outFields is None:
        outFields = '*'
    elif isinstance(outFields, list):
        outFields = ','.join(outFields)
    url = f"{base_url}{path}/query?where=1%3D1&outFields={outFields}&f=geojson"
    response = requests.get(url)
    geojson = json.loads(response.text)
    return geojson

def get_geojson_as_gp(base_url, path, outFields=None):
    if outFields is None:
        outFields = '*'
    elif isinstance(outFields, list):
        outFields = ','.join(outFields)
    url = f"{base_url}{path}/query?where=1%3D1&outFields={outFields}&f=geojson"
    df = gp.read_file(url, driver="GeoJSON")
    return df
    

Get subbasins

In [71]:
def gdf_to_ee(gdf):

    # Initialize an empty list to store features
    features = []

    # Iterate over each row in the GeoPandas DataFrame
    for index, row in gdf.iterrows():

        # Create a new Earth Engine feature from the row's geometry and properties
        feature = ee.Feature(ee.Geometry(row['geometry']), row.to_dict())

        # Append the feature to the list of features
        features.append(feature)

    # Create a new Earth Engine FeatureCollection from the list of features
    fc = ee.FeatureCollection(features)
    return(fc)

In [158]:
#start with subbasins

base_url = "https://gis.cityoftacoma.org/arcgis/rest/services/"
subbasins_path = "ES/SurfacewaterNetwork/MapServer/41" 
subbasins = get_geojson_as_gp(base_url,subbasins_path,outFields = ['BASINNAME','SUBBASIN'])
subbasins_geojson = get_geojson_as_json(base_url,subbasins_path)
subbasins.head()

 

,BASINNAME,SUBBASIN,geometry
0,WESTERN SLOPES,WS_03,"POLYGON ((-122.52683 47.25504, -122.52698 47.2..."
1,WESTERN SLOPES,WS_01,"POLYGON ((-122.51714 47.27441, -122.51758 47.2..."
2,FLETT CREEK,FL_03,"POLYGON ((-122.47503 47.20173, -122.47490 47.2..."
3,TIDEFLATS,TF_01,"POLYGON ((-122.42198 47.26224, -122.42210 47.2..."
4,NORTHEAST TACOMA,NE_03,"POLYGON ((-122.38348 47.29529, -122.38389 47.2..."


# Land Use

% Commercial 

% Industrial 

% Roads 

% Single Family Residential 

% Multifamily Residential 

% Parks & Undeveloped Land 

In [20]:
#ee.Initialize()
#!{'earthengine authenticate'}

In [21]:
# construct a FeatureCollection object from the json object
#fc = ee.FeatureCollection(subbasins)

In [22]:
# Get Land use layer 

landuse_path = "General/LandUseDesignations/MapServer/0" 
landuse = get_geojson_as_gp(base_url,landuse_path)
landuse.head()

,OBJECTID,LandUseDesignation,MixedUseCenter,CreatedBy,CreatedDate,LastEditor,LastEdited,GlobalID,SHAPE.STArea(),SHAPE.STLength(),geometry
0,1,Neighborhood Commercial,,GIS,1.589440e+12,JSHELL,1646899200000,{4DBA624E-52CB-4DD2-8EC3-F53406B03BDD},780055.492840,6999.852238,"POLYGON ((-122.51593 47.25542, -122.51596 47.2..."
1,2,Neighborhood Commercial,,GIS,1.589440e+12,JSHELL,1646899200000,{79CEDC72-94D6-48F1-8ED9-D61439CDE7BC},392833.682215,2976.539156,"POLYGON ((-122.49433 47.25599, -122.49433 47.2..."
2,3,Neighborhood Commercial,,GIS,1.589440e+12,JSHELL,1646812800000,{FC67B342-7357-4B29-9E8A-A6F56DCA69DF},664591.050547,6102.065086,"POLYGON ((-122.45586 47.25730, -122.45598 47.2..."
3,4,Neighborhood Commercial,,GIS,1.589440e+12,JSHELL,1646899200000,{A1650268-55B7-4B9F-8ABF-B57830203625},161966.320290,1813.323761,"POLYGON ((-122.51508 47.26058, -122.51572 47.2..."
4,5,Neighborhood Commercial,,GIS,1.589440e+12,JSHELL,1643356800000,{225DAC6B-6459-460E-8391-1337EBB7D345},130461.335699,1519.301520,"POLYGON ((-122.45770 47.26217, -122.45759 47.2..."


In [23]:
def overlay_and_summarize(x,by_x,by_y="SUBBASIN",y=subbasins): 
    x = x.to_crs('EPSG:3857')
    y = y.to_crs('EPSG:3857')
    shp_tmp = y.overlay(x, how='intersection')

    shp_tmp['intersected_area'] = shp_tmp.area
    shp_tmp = shp_tmp[[by_x,by_y,'intersected_area']]
    summary = shp_tmp.groupby([by_y,by_x]).sum()
    summary['percent'] = summary['intersected_area'] / summary.groupby(by_y)["intersected_area"].transform('sum')
    return(summary)

In [24]:
lu_summary = overlay_and_summarize(landuse,'LandUseDesignation')
lu_summary.style.format({'percent': '{:.2%}'.format})
lu_summary.head

<bound method NDFrame.head of                                       intersected_area   percent
SUBBASIN LandUseDesignation                                     
FL_01    Crossroads Mixed-Use Center      3.664704e+03  0.000832
         General Commercial               1.859721e+05  0.042233
         Low-Scale Residential            2.572084e+06  0.584108
         Major Institutional Campus       5.505233e+05  0.125021
         Mid-Scale Residential            5.342479e+05  0.121325
...                                                ...       ...
WS_04    Low-Scale Residential            3.308445e+06  0.682731
         Mid-Scale Residential            6.819940e+05  0.140736
         Neighborhood Commercial          9.638384e+03  0.001989
         Parks and Open Space             4.485093e+05  0.092554
         Shoreline                        3.679924e+05  0.075939

[384 rows x 2 columns]>

In [26]:
lu_summary

intersected_area   percent
SUBBASIN LandUseDesignation                                     
FL_01    Crossroads Mixed-Use Center      3.664704e+03  0.000832
         General Commercial               1.859721e+05  0.042233
         Low-Scale Residential            2.572084e+06  0.584108
         Major Institutional Campus       5.505233e+05  0.125021
         Mid-Scale Residential            5.342479e+05  0.121325
...                                                ...       ...
WS_04    Low-Scale Residential            3.308445e+06  0.682731
         Mid-Scale Residential            6.819940e+05  0.140736
         Neighborhood Commercial          9.638384e+03  0.001989
         Parks and Open Space             4.485093e+05  0.092554
         Shoreline                        3.679924e+05  0.075939

[384 rows x 2 columns]

In [28]:
#lu_summary.pivot(index = 0, columns='LandUseDesignation', values='percent')

lu_summary_df = lu_summary.reset_index()
pivot_table = lu_summary_df.pivot(index='SUBBASIN', columns='LandUseDesignation', values='percent')
pivot_table

In [31]:
pivot_table.to_csv('exported_data/landuse.csv')

# Land Cover

% Forest
% Pasture
%Landscape
%Effective Impervious Surface

In [49]:
import ee 
ee.Initialize()
landcover_layer = ee.Image("projects/ee-swhm/assets/production_layers/HSPF_Land_Cover_Type") 
fc = ee.FeatureCollection(subbasins_geojson)

In [56]:
# Create an image with pixel area and land cover layer
img = ee.Image.pixelArea().addBands(landcover_layer)

# Filter the watersheds table by geometry
geometry =     ee.Geometry.Polygon(
        [[[-122.51117300651649, 47.27135819591651],
          [-122.51117300651649, 47.216352555552646],
          [-122.45006155632117, 47.216352555552646],
          [-122.45006155632117, 47.27135819591651]]]);


In [129]:
# Define the summarize_groups function
def summarize_groups(feature):
    return feature.select(['SUBBASIN','groups'],retainGeometry=False)
# Reduce the image by watersheds and summarize the groups
lc_summary = img.reduceRegions(
    collection=fc,
    reducer=ee.Reducer.sum().group(
        groupField=1,
        groupName='class value'
    ),
    scale=10
).map(summarize_groups)

data = lc_summary.getInfo()
import json



# Access properties of each feature
for feature in data['features']:
    groups = feature['properties']['groups']
    
    print(groups)
# Print the land cover summary

[{'class value': 0, 'sum': 403702.4926592059}, {'class value': 1, 'sum': 552408.9458083506}, {'class value': 2, 'sum': 517293.7250080183}, {'class value': 3, 'sum': 44620.25246710685}, {'class value': 4, 'sum': 452708.64505468635}, {'class value': 5, 'sum': 794207.7051418227}]
[{'class value': 0, 'sum': 1805662.2313714137}, {'class value': 1, 'sum': 334657.15909343056}, {'class value': 2, 'sum': 293116.05679393077}, {'class value': 3, 'sum': 43867.26018530151}, {'class value': 4, 'sum': 232029.68932372457}, {'class value': 5, 'sum': 386715.0641378965}]
[{'class value': 0, 'sum': 750034.2561410052}, {'class value': 1, 'sum': 1261949.1943272313}, {'class value': 2, 'sum': 1431501.4796279925}, {'class value': 3, 'sum': 8193.493944833794}, {'class value': 4, 'sum': 1063263.2957162901}, {'class value': 5, 'sum': 2349192.3468385586}]
[{'class value': 0, 'sum': 5131.471637860466}, {'class value': 1, 'sum': 264080.1105069329}, {'class value': 2, 'sum': 52216.81205303715}, {'class value': 3, 's

In [130]:
data

{'type': 'FeatureCollection',
 'columns': {},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '0',
   'properties': {'SUBBASIN': 'WS_03',
    'groups': [{'class value': 0, 'sum': 403702.4926592059},
     {'class value': 1, 'sum': 552408.9458083506},
     {'class value': 2, 'sum': 517293.7250080183},
     {'class value': 3, 'sum': 44620.25246710685},
     {'class value': 4, 'sum': 452708.64505468635},
     {'class value': 5, 'sum': 794207.7051418227}]}},
  {'type': 'Feature',
   'geometry': None,
   'id': '1',
   'properties': {'SUBBASIN': 'WS_01',
    'groups': [{'class value': 0, 'sum': 1805662.2313714137},
     {'class value': 1, 'sum': 334657.15909343056},
     {'class value': 2, 'sum': 293116.05679393077},
     {'class value': 3, 'sum': 43867.26018530151},
     {'class value': 4, 'sum': 232029.68932372457},
     {'class value': 5, 'sum': 386715.0641378965}]}},
  {'type': 'Feature',
   'geometry': None,
   'id': '2',
   'properties': {'SUBBASIN': 'FL_03',
    'groups

In [148]:
# Normalize the "groups" property into a DataFrame
df = pd.json_normalize(data, record_path='features', 
                       #record_prefix='properties.'
                       errors='ignore')

class_df = pd.DataFrame()
for i in range(df.shape[0]):
    groups = df.loc[i, 'properties.groups']
    temp_df = pd.json_normalize(groups)
    temp_df = temp_df.set_index('class value').T
    temp_df.index = [i]
    class_df = pd.concat([class_df, temp_df], axis=0, sort=False)


# Merge the class DataFrame with the original DataFrame
df = pd.merge(df, class_df, left_index=True, right_index=True)

# Drop unnecessary columns
df = df.drop(columns=['geometry', 'properties.groups'])

new_cols = {
    0:  "Forest/Trees",
    1: "Pasture",
    2: "Grass",
    3: "Water",
    4: "Impervious-roof",
    5: "Impervious-nonRoof"
}
df_named = df.rename(columns=new_cols).drop(['id','type'], axis=1)



In [149]:
df_named

,properties.SUBBASIN,Forest/Trees,Pasture,Grass,Water,Impervious-roof,Impervious-nonRoof
0,WS_03,4.037025e+05,5.524089e+05,5.172937e+05,44620.252467,4.527086e+05,7.942077e+05
1,WS_01,1.805662e+06,3.346572e+05,2.931161e+05,43867.260185,2.320297e+05,3.867151e+05
2,FL_03,7.500343e+05,1.261949e+06,1.431501e+06,8193.493945,1.063263e+06,2.349192e+06
3,TF_01,5.131472e+03,2.640801e+05,5.221681e+04,91419.085543,1.970301e+05,4.381448e+05
4,NE_03,1.158250e+06,5.655599e+05,1.001179e+06,60787.184197,5.687487e+05,6.035520e+05
...,...,...,...,...,...,...,...
62,LP_05,6.904346e+05,3.634568e+05,5.792514e+05,NaN,3.261237e+05,5.956801e+05
63,FL_08,6.614563e+04,1.893979e+05,9.621925e+04,NaN,1.838613e+05,2.912730e+05
64,FL_01,3.422967e+05,3.704481e+05,4.061405e+05,NaN,3.599603e+05,5.513574e+05
65,TF_04,1.753357e+03,1.314559e+05,6.574342e+04,11722.799520,1.078701e+05,1.414221e+06


In [154]:
df_named

# calculate the total area for each row
total_area = df_named.sum(axis=1)

# divide each land cover type by the total area to get the percentage
df_percent = df_named.iloc[:, 1:].div(total_area, axis=0) 

# concatenate the ID column with the percentage columns
df_percent.insert(0, 'SUBBASIN', df['properties.SUBBASIN'])

# print the resulting DataFrame
df_percent.to_csv('exported_data/landcover.csv')

/var/folders/yn/15903z7124l3th5fm7wg0lgw0000gn/T/ipykernel_20496/3915731589.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  total_area = df_named.sum(axis=1)


# Habitat
Tree Canopy % Cover in buffers (varies for stream type)


Biodiversity Cooridor
150-foot Buffer % vegetated
75-foot Buffer % vegetated
25-foot Buffer % vegetated

# Water Quality
303d (listing) 
TMDL (y/n)

In [32]:
attains_url = 'https://gispub.epa.gov/arcgis/rest/services/OW/ATTAINS_Assessment/MapServer/1' 

In [33]:
feature = subbasins.iloc[0]
geom = list(feature.geometry.exterior.coords)
coord_strings = [f"{x}, {y}" for x, y in geom]
coord_strings

['-122.52683047424546, 47.25504204954043',
 '-122.52698273419371, 47.25520835004718',
 '-122.52708039055457, 47.25547499482117',
 '-122.52704599948555, 47.25572407325478',
 '-122.52687955918285, 47.25595558523543',
 '-122.5265077489281, 47.25616087123752',
 '-122.52616811234994, 47.25644510653198',
 '-122.52616251954599, 47.25668374170972',
 '-122.52627663791104, 47.25699979519835',
 '-122.52647979210587, 47.25735404711795',
 '-122.52672296723276, 47.25760820790029',
 '-122.5268056624389, 47.257865176337695',
 '-122.52693175153361, 47.25811144632104',
 '-122.52692353700667, 47.25828055395407',
 '-122.52663812281496, 47.25845451182343',
 '-122.52656220874924, 47.25876394963658',
 '-122.52648216980856, 47.25896413178532',
 '-122.52638754055458, 47.25916456729309',
 '-122.52638419787321, 47.259462798058664',
 '-122.5264040603891, 47.259881035010714',
 '-122.52639758198893, 47.260607355313056',
 '-122.5263969458543, 47.26067877799427',
 '-122.526393628453, 47.26105079332399',
 '-122.526525

In [34]:
tacoma_bbox ="-122.66645777315975, 47.066727220416695, -122.24485743136287, 47.353125316075044"

In [35]:
# From 
# https://gis.stackexchange.com/questions/427434/query-feature-service-on-esri-arcgis-rest-api-with-python-requests
import urllib.parse

attains_url = 'https://gispub.epa.gov/arcgis/rest/services/OW/ATTAINS_Assessment/MapServer/1/query?'

params = {
    'geometry':tacoma_bbox,
    'geometryType': 'esriGeometryEnvelope',
    'inSR': '4326',
    'returnGeometry': 'true', 
    'outFields': 'assessmentunitidentifier,assessmentunitname,tas303d,reportingcycle,waterbodyreportlink,ircategory',
    'f': 'geojson'
}

url_final = attains_url + urllib.parse.urlencode(params)
url_final

'https://gispub.epa.gov/arcgis/rest/services/OW/ATTAINS_Assessment/MapServer/1/query?geometry=-122.66645777315975%2C+47.066727220416695%2C+-122.24485743136287%2C+47.353125316075044&geometryType=esriGeometryEnvelope&inSR=4326&returnGeometry=true&outFields=assessmentunitidentifier%2Cassessmentunitname%2Ctas303d%2Creportingcycle%2Cwaterbodyreportlink%2Circategory&f=geojson'

In [39]:
 wq_lines = gp.read_file(url_final,driver="GeoJSON")

In [43]:
#get intersecting water quality lines 

water_quality = gp.overlay(wq_lines, subbasins, how='intersection')


In [47]:
gdf_no_geom = water_quality.drop(columns='geometry')

# Save as CSV without geometry column
gdf_no_geom.to_csv('exported_data/waterquality.csv', index=False)

# of discharge points (Outfalls)
# of Ditches
# of Culvert Crossings


In [ ]:
ditches = "33" 

In [178]:
discharge_pts_path = "ES/SurfacewaterNetwork/MapServer/24"
ditches_path = "ES/SurfacewaterNetwork/MapServer/30"

In [182]:
discharge_pts_path = "ES/SurfacewaterNetwork/MapServer/24" 
discharge_pts = get_geojson_as_gp(base_url,discharge_pts_path)
ditches = get_geojson_as_gp(base_url,ditches_path)

In [165]:
urban_heat = "https://gis.cityoftacoma.org/arcgis/rest/services/ES/UrbanHeatIslandIndex/MapServer/0"
subbasins.head

<bound method NDFrame.head of            BASINNAME SUBBASIN  \
0     WESTERN SLOPES    WS_03   
1     WESTERN SLOPES    WS_01   
2        FLETT CREEK    FL_03   
3          TIDEFLATS    TF_01   
4   NORTHEAST TACOMA    NE_03   
..               ...      ...   
62    LOWER PUYALLUP    LP_05   
63       FLETT CREEK    FL_08   
64       FLETT CREEK    FL_01   
65         TIDEFLATS    TF_04   
66       FLETT CREEK    FL_10   

                                             geometry  
0   POLYGON ((-122.52683 47.25504, -122.52698 47.2...  
1   POLYGON ((-122.51714 47.27441, -122.51758 47.2...  
2   POLYGON ((-122.47503 47.20173, -122.47490 47.2...  
3   POLYGON ((-122.42198 47.26224, -122.42210 47.2...  
4   POLYGON ((-122.38348 47.29529, -122.38389 47.2...  
..                                                ...  
62  POLYGON ((-122.39694 47.22092, -122.39714 47.2...  
63  POLYGON ((-122.45568 47.18457, -122.45606 47.1...  
64  POLYGON ((-122.49178 47.25034, -122.49200 47.2...  
65  POLYGON (

In [185]:
# Group by the boundary name and count the number of facilities
discharge_pt_count = discharge_pts.groupby('SUBBASIN').size().reset_index(name='facility_count')
ditches_count = ditches.groupby('SUBBASIN').size().reset_index(name='ditches_count')
ditches_count.to_csv('exported_data/ditches.csv')
discharge_pt_count.to_csv('exported_data/discharge_pts.csv')